# Nuevos planes.

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.
Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos. 

Desarrolla un modelo con la mayor exactitud posible, el umbral de exactitud es 0.75. 

1.- Segmentaremos los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.

2.- Investiga la calidad de diferentes modelos cambiando los hiperparámetros. 

3.- Comprueba la calidad del modelo usando el conjunto de prueba.

## Revisamos los datos.

In [1]:
#Importamos las librerias que necesitamos.

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

In [2]:
#Cargamos nuestro DataFrame
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [4]:
# Revisamos las columnas del DataFrame
df.columns

Index(['calls', 'minutes', 'messages', 'mb_used', 'is_ultra'], dtype='object')

In [5]:
#Revisamos que no tenga valores nulos
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [6]:
#Revisamos que no tenga valores duplicados
df.duplicated().sum()

0

## Conclusion.
Ya que tenemos nuestro DataFrame observamos que esta listo para comenzar a desarrollar nuestro modelo.

## Creando nuestro modelo.

Para crear el modelo primero vamos a divisir nuestro DataFrame en conjuntos de entrenamiento, validacion y prueba, para esto establecimos que los conjuntos tendran 60%, 20% y 20% del totl de los datos respectivamente ya que parecio correcto por la cantidad de datos que se manejan.

Para crear los modelos usaremos bucles que nos digan cuales fueron los hiperparametros que nos dieron los mejores resultados y en base a estos resultados con los conjuntos de entrenamiento y validacion escogeremos uno para pasarle ahora el conjunto de prueba.

Ya que tengamos los resultados calcularemos la exactitud del modelo.

In [7]:
# Dividimos el DataFrame en conjuntos de entrenamiento, validacion y prueba.
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.20, random_state=12345)

features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train, test_size=0.25, random_state=12345)

In [8]:
# Revisamos los DataFrames resultantes tengan la cantidad de datos necesarios con tengan los datos correspondientes.
features_train.shape

(1928, 4)

In [9]:
features_valid.shape

(643, 4)

In [10]:
features_test.shape

(643, 4)

In [11]:
features_train.head()

,calls,minutes,messages,mb_used
2656,30.0,185.07,34.0,17166.53
823,42.0,290.69,77.0,21507.03
2566,41.0,289.83,15.0,22151.73
1451,45.0,333.49,50.0,17275.47
2953,43.0,300.39,69.0,17277.83


In [12]:
features_valid.head()

,calls,minutes,messages,mb_used
2699,71.0,512.53,27.0,15772.68
242,183.0,1247.04,150.0,29186.41
2854,34.0,246.06,31.0,8448.76
1638,63.0,468.66,0.0,11794.34
1632,4.0,19.85,28.0,13107.42


In [13]:
features_test.head()

,calls,minutes,messages,mb_used
1415,82.0,507.89,88.0,17543.37
916,50.0,375.91,35.0,12388.40
1670,83.0,540.49,41.0,9127.74
686,79.0,562.99,19.0,25508.19
2951,78.0,531.29,20.0,9217.25


In [14]:
target_train.head()

2656    0
823     0
2566    0
1451    0
2953    0
Name: is_ultra, dtype: int64

In [15]:
target_valid.head()

2699    0
242     1
2854    0
1638    0
1632    0
Name: is_ultra, dtype: int64

In [16]:
target_test.head()

1415    1
916     0
1670    0
686     1
2951    0
Name: is_ultra, dtype: int64

## Modelo de arbol de desiciones.

In [17]:
# Creamos modelo de arbol de desiciones.
best_score = 0
best_depth = 0

for depth in range(1,11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    
    score = model.score(features_valid, target_valid)
    
    if score > best_score:
        best_score = score
        best_depth = depth

print('max_depth = {}: {}'.format(best_depth, best_score))
   

max_depth = 7: 0.7744945567651633


## Conclusion.

Este modelo nos da una exactitud del 77% a una profundidad de 7, en este caso se probo el modelo con una profundidad maxima de 50 y no mejoro siendo 7 siempre el mejor resultado.

## Modelo de bosque aleatorio.

In [18]:
# Creamos modelo de bosque aleatorio.
best_score=0
best_est =0
best_depth = 0

for est in range(1, 41, 10):
    for depth in range(1,11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        score = model.score(features_valid, target_valid)
    
        if score > best_score:
            best_score = score
            best_est = est
            best_depth = depth
    
print('Accuracy del mejor modelo de validacion (score= {}, n_estimators= {}, depth= {}'.format(best_score, best_est, best_depth))

Accuracy del mejor modelo de validacion (score= 0.7993779160186625, n_estimators= 36, depth= 10


## Conclusion

De la misma manera en este modelo se probaron varios hiperparametros con n_estimators = 40 y max_depth = 10 pero fueron 36 y 10 respectivamente los que nos dieron una exactitud de 80%.

NOTA: Los hiperparametros se dejaron en estos valores maximos para la prueba para que la compilacion no tardara tanto.

## Modelo de regresion logistica

In [19]:
# Creamos modelo de regresion logistica.
model = LogisticRegression(random_state=12345, solver = 'liblinear')
model.fit(features_train, target_train)
score_train = model.score(features_train, target_train)
score_valid = model.score(features_valid, target_valid)

print('Accuracy del modelo de regresion logistica en el conjunto de entrenamiento', score_train)
print('Accuracy del modelo de regresion logistica en el conjunto de validacion', score_valid)

Accuracy del modelo de regresion logistica en el conjunto de entrenamiento 0.7422199170124482
Accuracy del modelo de regresion logistica en el conjunto de validacion 0.7293934681181959


## Conclusion.

Este es el modelo que peores resultados nos ha dado ya que primero no se encuentra en el umbral que debemos alcanzar sino que los resultados conparados con el conjunto de entrenamiento fueron menores por lo que queda descartado completamente.

## Modelo de prueba.

Despues de observar los resultados de nuestros tres modelos tenemos al bosque aleatorio con la exactitud mas alta por lo que pasaremos a este modelo el cojunto de prueba con los hiperparametros de n_estimators = 20 y max_depth = 15 que fueron los que nos dieron los mejores resultados.

In [20]:
# Pasamos datos de prueba al modelo con mejores resultados.
model = RandomForestClassifier(random_state=12345, n_estimators=36, max_depth=10)
model.fit(features_train, target_train)
answers = model.predict(features_test)
predictions = pd.Series(answers)
predictions

0      0
1      0
2      0
3      0
4      0
      ..
638    0
639    0
640    0
641    1
642    0
Length: 643, dtype: int64

In [21]:
# Comprobamos la exactitud del modelo con los datos de prueba.
accuracy = accuracy_score(target_test, predictions)
accuracy

0.8009331259720062

In [22]:
emc = mean_squared_error(target_test, predictions)
emc

0.19906687402799378

In [23]:
remc = mean_squared_error(target_test, predictions, squared=False)
remc

0.44616910922652836

## Conclusiones finales.

Despues de crear tres modelos uno de arbol de desiciones, bosque aleatorio y regresion lineal elegimos al bosque aleatorio con ciertos hiperparametros como el que nos daba el mejor resultado con 78% y al pasarle los datos de prueba nos da una exactitud del 80%.

NOTA: En cuanto a la prueba de de cordura del modelo realice la unica que vimos en el curso sin embargo no estoy seguro de que sea la correcta para evaluar los datos y resultados que obtuvimos y aunque investigue otras como el coeficiente de determinacion r2, error absoluto maximo y medio no encontre una de la cual pudiera interpretar correctamente los resultados.